#직방 송파 > 송파동 > 단지목록 > url decode(구글 > 한글변환)

https://apis.zigbang.com/apt/locals/prices/on-danjis?minPynArea=10평이하&maxPynArea=60평대이상&geohash=wydmk #선택한것

https://apis.zigbang.com/v2/search?leaseYn=N&q=송파&serviceType=아파트 #검색한것

In [25]:
import requests
region = '송파'
kind = '아파트'




url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={region}&serviceType={kind}'

url

res = requests.get(url)

data = res.json()['items'][0]
data
lat, lng = data['lat'], data['lng']
lat, lng

(37.514591217041, 127.105865478516)

위 경도 데이터

In [26]:
!pip install geohash2

  Preparing metadata (setup.py) ... done
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15543 sha256=54ab3d2b8e104f24cc26e433267899e41eb9d74df1ecb7c80e457fefb231f979
  Stored in directory: /Users/ruben/Library/Caches/pip/wheels/f6/7c/c4/1b3c6fea0ebc53bf730dc86bbee7a713d501455dfb4c1f0623
Successfully built geohash2


In [45]:
import geohash2
geohash = geohash2.encode(lat, lng, precision=5) #위도, 경도 , 정밀도(작게:0 세계지도, 17 건물)
geohash

'wydmk'

In [46]:
url2 = f'https://apis.zigbang.com/apt/locals/prices/on-danjis?minPynArea=10평이하&maxPynArea=60평대이상&geohash={geohash}'

url2

'https://apis.zigbang.com/apt/locals/prices/on-danjis?minPynArea=10평이하&maxPynArea=60평대이상&geohash=wydmk'

In [ ]:
import requests
import pandas as pd

res = requests.get(url2)
res.text

data = res.json()['filtered']
df = pd.DataFrame(data)
df.price

In [58]:
pd.options.display.max_columns = len(df.columns)
pd.set_option('display.max_columns', None)
df['max_price'] =  df['price'].apply(lambda x: x['sales']['max'] if isinstance(x,dict) else None)
df['perArea_price'] =  df['price'].apply(lambda x: x['sales']['perArea'] if isinstance(x,dict) else None)
# print(df.max_price)
# print(df.perArea_price)

df.drop('price', axis=1, inplace=True)

In [59]:
df.columns

Index(['id', 'name', 'lat', 'lng', '총세대수', '사용승인일', '서비스구분', 'real_type',
       'sido', 'gugun', 'dong', 'image', 'view_sources', 'is후분양', 'max_price',
       'perArea_price'],
      dtype='object')

In [66]:
col = ['id', 'name', 'lat', 'lng', '총세대수', 
        'gugun', 'dong', 'max_price','perArea_price']
df = df[col]

df['lat'] = df['lat'].apply(lambda x: int(x * 100)/100)
df['lng'] = df['lng'].apply(lambda x: int(x * 100)/100)
df

,id,name,lat,lng,총세대수,gugun,dong,max_price,perArea_price
0,1625,수서1단지,37.49,127.09,2934,강남구,일원동,146000.0,4465.0
1,14453,트리지움,37.50,127.09,3696,송파구,잠실동,345000.0,6871.0
2,6232,갤러리아팰리스,37.51,127.09,741,송파구,잠실동,610000.0,4960.0
3,10591,레이크팰리스,37.50,127.09,2678,송파구,잠실동,328000.0,6371.0
4,1069,잠실주공5단지,37.51,127.09,3930,송파구,잠실동,281000.0,7553.0
...,...,...,...,...,...,...,...,...,...
149,58786,진넥스베르디엠,37.49,127.13,49,송파구,가락동,79000.0,3568.0
150,10007,신일유토빌,37.52,127.13,65,강동구,성내동,87000.0,2738.0
151,80704,힐스테이트e편한세상문정,37.47,127.13,1265,송파구,문정동,NaN,NaN
152,1230,프라자,37.49,127.13,672,송파구,가락동,156000.0,3202.0


In [15]:
import requests
import pandas as pd
import geohash2


def get_apt_data(region = '송파'):
    kind = '아파트'
    
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={region}&serviceType={kind}'
    
    res = requests.get(url)
    
    data = res.json()['items'][0]
    
    lat, lng = data['lat'], data['lng']
    
    geohash = geohash2.encode(lat, lng, precision=5) #위도, 경도 , 정밀도(작게:0 세계지도, 17 건물)
    
    url2 = f'https://apis.zigbang.com/apt/locals/prices/on-danjis?minPynArea=10평이하&maxPynArea=60평대이상&geohash={geohash}'
    
    res = requests.get(url2)
    
    data = res.json()['filtered']
    df = pd.DataFrame(data)
    
    pd.options.display.max_columns = len(df.columns)
    pd.set_option('display.max_columns', None)
    df['max_price'] =  df['price'].apply(lambda x: x['sales']['max'] if isinstance(x,dict) else None)
    df['perArea_price'] =  df['price'].apply(lambda x: x['sales']['perArea'] if isinstance(x,dict) else None)
    
    
    df.drop('price', axis=1, inplace=True)
    
    col = ['id', 'name', 'lat', 'lng', '총세대수', 'gugun', 'dong', 'max_price','perArea_price']
    df = df[col]
    
    df['lat'] = df['lat'].apply(lambda x: int(x * 100)/100)
    df['lng'] = df['lng'].apply(lambda x: int(x * 100)/100)
    return(df)


df = get_apt_data('서초동')
df

,id,name,lat,lng,총세대수,gugun,dong,max_price,perArea_price
0,4250,현대동궁,37.50,127.00,224,서초구,반포동,165000.0,5692.0
1,80055,반포르엘2차,37.50,127.00,280,서초구,잠원동,360000.0,8784.0
2,74508,반포르엘,37.50,127.00,596,서초구,잠원동,410000.0,8899.0
3,6474,한남힐스테이트,37.52,127.00,283,용산구,한남동,251000.0,4848.0
4,17086,서초두산위브2,37.48,127.00,47,서초구,서초동,225000.0,4571.0
...,...,...,...,...,...,...,...,...,...
265,6201,한신오페라하우스1차,37.51,127.04,63,강남구,청담동,211000.0,4715.0
266,18932,청담3차e편한세상,37.51,127.04,49,강남구,청담동,229000.0,5079.0
267,51409,강남센트럴아이파크,37.50,127.04,499,강남구,역삼동,480000.0,8708.0
268,11503,역삼I′PARK1차,37.49,127.04,541,강남구,역삼동,363000.0,6890.0


In [74]:
q1 = df[(df['총세대수'] > 200) & (df['perArea_price'] < 4000)]
q1

,id,name,lat,lng,총세대수,gugun,dong,max_price,perArea_price
57,6064,서초이오빌,37.48,127.01,462,서초구,서초동,62000.0,2402.0
76,6155,서초리시온,37.49,127.01,293,서초구,서초동,83000.0,2771.0
92,6156,서초대우아이빌,37.49,127.01,215,서초구,서초동,54000.0,2200.0
155,7153,삼성쉐르빌II,37.49,127.02,365,서초구,서초동,92000.0,2427.0
159,38644,마일스디오빌,37.51,127.02,260,강남구,논현동,38000.0,2533.0
180,4010,역삼한스빌,37.49,127.03,216,강남구,역삼동,46000.0,2238.0
242,5944,디오빌역삼,37.50,127.04,457,강남구,역삼동,116000.0,3042.0
251,5017,한화진넥스빌,37.50,127.04,294,강남구,역삼동,52000.0,2446.0


In [2]:
import  pygsheets
gc = pygsheets.authorize(client_secret="client_secret.json", credentials_directory="./path/to/credentials")

Please go to this URL and finish the authentication flow: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=269740743783-2ju5uabh4h6c3te9pdtba8v9rtcpvf0q.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=sKhfjIXJV7CqoqYcrZUYQeFSP0wvla&prompt=consent&access_type=offline


Enter the authorization code:  4/1AfJohXk8CQ7biEaLNV_yiN8FyGwdZV1OEiJsQ_xBft96exdGxhpQqjxSS9E


Unable to save the credentials to file-system


In [3]:
my_worksheet = gc.create("natural_excel1")#create메서드로 스프레드시트 문서 생성

In [4]:
import seaborn as sns
iris_df = sns.load_dataset("iris")
iris_df.tail(1)

,sepal_length,sepal_width,petal_length,petal_width,species
149,5.9,3.0,5.1,1.8,virginica


In [5]:
iris_sheet = my_worksheet.add_worksheet("iris222")

In [6]:
iris_sheet.set_dataframe(iris_df, "A1")

In [8]:
df_download = iris_sheet.get_as_df()
df_download

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [9]:
iris_sheet.update_value('A2', 'hello, python!')

In [10]:
my_worksheet[1]

<Worksheet 'iris222' index:1>

In [11]:
my_worksheet.del_worksheet(my_worksheet[1])

In [12]:
work_list = my_worksheet.worksheets()
work_list

[<Worksheet '시트1' index:0>]

In [16]:
addrs = ['서초동', '과천', '삼평동','주엽동','서귀포동']
for ele in addrs:
    print(ele)
    df = get_apt_data(ele)
    sheet = my_worksheet.add_worksheet(ele)
    sheet.set_dataframe(df,'A1')

서초동
과천
삼평동
주엽동
서귀포동


In [ ]:
<div id="wrap" class="dss">data 1</div>
<p id="txt" class="dss-txt no1">data 2</p>
<span class="dss-txt no2" val="d3">data 3</span>
<span id="da4" val="d4">data 4</span>
<span class="no5">data 5</span>


<div>
    <span>data 0</span>
    <p class="ds dss1">data 1</p>
    <p class="ds dss2">data 2</p>################################# 3번째
    <p class="ds dss3">data 3</p>
    <p class="ds dss4">data 4</p>
    <div class="wrap">
        <p class="ds dss1">data 1</p>
        <p class="ds dss2">data 2</p>
        <p class="ds dss3">data 3</p>#################################3번째
        <p class="ds dss4">data 4</p>
    </div>
</div>
<div></div>
<div class="ds">data 5</div



In [19]:
from IPython.display import HTML

html_code4 = '''
<p>지치지 않기</p>
'''
HTML(html_code4)

In [27]:
from IPython.display import HTML

html_code1 = '''
<div class="wrapper">
    <button  class="bt no1" type="button" >HTML 1</button>
    <button  class="bt no2" type="button">HTML 2</button>
</div>
'''
html_code2 = '''
<div class="wrapper">
    <button id="btn-1" class="bt no1" type="button" >HTML 1</button>
    <button id="btn-2" class="bt no2" type="button" style="color:red;">HTML 2</button>
</div>
'''
html_code4 = '''
<p>지치지 않기123</p>
</div>
'''
html_code6 = '''
<div>
    <p>html 1</p>
    <p>html 2</p>
</div>
<div>
    <p>html 3</p>
</div>
'''
html_code7 = '''
<table>
    <caption>테이블 제목</caption>
    <thead>
        <tr>
            <th>코드</th>
            <th>회사명</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>0001</td>
            <td>APPLE</td>
        </tr>
        <tr>
            <td>0002</td>
            <td>테슬라</td>
        </tr>
    </tbody>
</table>


'''






HTML(html_code4)
HTML(html_code7)

코드,회사명
0001,APPLE
0002,테슬라


In [ ]:
body > div.container-doc > main > section > div > div.content-article > div.box_g.box_news_issue > ul > li:nth-child(1) > div > div > strong > a

In [30]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
res = requests.get("https://news.daum.net/")
res.text
#보기 좋은 html상태로 변환시도
soup = BeautifulSoup(res.content, "html.parser")
soup


<!DOCTYPE html>

<html class="os_unknown none unknown version_0" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="always" name="referrer">
<meta content="Daum 뉴스" property="og:author"/>
<meta content="다음뉴스" property="og:site_name"/>
<meta content="홈" property="og:title"/>
<meta content="https://t1.daumcdn.net/media/img-media/mobile/meta/news.png" property="og:image"/>
<meta content="다음뉴스" property="og:description"/>
<meta content="https://news.daum.net/" property="og:url"/>
<link href="https://m2.daumcdn.net/img-media/2010ci/Daum_favicon.ico" rel="shortcut icon"/>
<title>홈 | 다음뉴스</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="//t1.daumcdn.net/media/kraken/news/402cee8/style.css.merged.css" rel="stylesheet" type="text/css">
<link href="//t1.daumcdn.net/media/kraken/news/402cee8/calendar.css.merged.css" rel="stylesheet" type="text/css"/>
<!--[if lte IE 8]>
<script src="https://m2.daumcdn.net/svc/original/U0301/cssjs/JSON-js/fc535e9cc8/json2.min.js"></

In [38]:
elements = soup.select('body > div.container-doc > main > section > div > div.content-article > div.box_g.box_news_issue > ul > li:nth-child(1) > div > div > strong > a')
len(elements)
elements = soup.select('div.content-article > div.box_g.box_news_issue > ul > li')
len(elements)
elements[0]

<li>
<div class="item_issue" data-tiara-layer="headline1">
<a class="wrap_thumb" data-tiara-custom="contentUniqueKey=hamny-20240129162306968&amp;clusterId=5889471,5590543,5150095&amp;clusterTitle=국제,[언론사픽] 주요뉴스,섹션 실시간뉴스-국제&amp;keywordType=NONE,NONE,NONE" data-tiara-id="20240129162306968" data-tiara-layer="article_main" data-tiara-ordnum="1" data-tiara-type="harmony" href="https://v.daum.net/v/20240129162306968">
<img alt="" class="thumb_g" src="https://img1.daumcdn.net/thumb/S96x60ht.u/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fnews%2F202401%2F29%2Fhankooki%2F20240129162309589mloz.jpg&amp;scode=media" 관광객="" 군마현지사,="" 뒤에선="" 유치="" 조선인="" 철거="" 추도비="" 한국="" 행보"=""/>
</a>
<div class="cont_thumb">
<span class="info_thumb">
<span class="logo_cp">
<img alt="한국일보" class="thumb_g" onerror="this.style.display='none';" src="https://t1.daumcdn.net/media/news/news2016/cp/cp_hankooki.gif"/>
</span>
<span class="txt_category">국제</span>
</span>
<strong class="tit_g">
<a class="link_txt" data-tiara-custom

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
html_string = '''
<div id="wrap" class="dss">data 1</div>
<p id="txt" class="dss-txt no1">data 2</p>
<span class="dss-txt no2" val="d3">data 3</span>
<span id="da4" val="d4">data 4</span>
<span class="no5">data 5</span>
<a href="https://v.daum.net/v/20240130105016433" class="link_txt">“난임 시술 지원비 신청할 때, 등본 안 떼가도 된다” 구비서류 ‘제로화’ 추진</a>
'''
soup2 =  BeautifulSoup(html_string, 'html.parser') #html >> BeautifulSoup 객체로 만들어짐
elements = soup2.select_one('a').get('href')
print(elements)
number = re.search(r'|d+', elements).group()
print(number)
html_string2 = '''<div class="wrap-1">
    <h5>inner 1<h5>
    <div class="wrap-2">
        <h5>inner 2<h5>
    </div>
</div>'''


https://v.daum.net/v/20240130105016433

